In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

# ---------------------------------
# 1. КОНФІГУРАЦІЯ
# ---------------------------------

DATA_DIR = 'learning/data_classifier_mega'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')

# Параметри
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 64 # Можна збільшити (128, 256), якщо дозволяє VRAM
COLOR_MODE = 'grayscale' # Важливо!
INPUT_SHAPE = (IMAGE_SIZE[0], IMAGE_SIZE[1], 1) # (64, 64, 1) для grayscale

# ---------------------------------
# 2. ЗАВАНТАЖЕННЯ ДАНИХ (ВИПРАВЛЕНО)
# ---------------------------------
print("--- Завантаження даних ---")

# Використовуємо 'image_dataset_from_directory' - це найефективніший спосіб
train_dataset = image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode=COLOR_MODE,
    label_mode='int' # Класи будуть числами (0, 1, 2...)
)

valid_dataset = image_dataset_from_directory(
    VALID_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode=COLOR_MODE,
    label_mode='int'
)

# !!! ВИПРАВЛЕННЯ ТУТ !!!
# Зберігаємо назви класів ДО оптимізації
class_names = train_dataset.class_names
print("--- Дані успішно завантажено ---")
print(f"Класи: {class_names}") # Друкуємо збережену змінну

# Оптимізація завантаження даних
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# ---------------------------------
# 3. СТВОРЕННЯ МОДЕЛІ (CNN)
# ---------------------------------

# Проста, але потужна архітектура CNN
model = Sequential([
    # 1. Шар нормалізації (переводить пікселі 0-255 в 0-1)
    Rescaling(1./255, input_shape=INPUT_SHAPE),

    # 2. Блок згортки 1
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # 3. Блок згортки 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # 4. Блок згортки 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # 5. "Випрямлення" даних
    Flatten(),

    # 6. Повнозв'язний шар
    Dense(128, activation='relu'),
    Dropout(0.5), # Dropout для боротьби з перенавчанням

    # 7. Вихідний шар (10 нейронів - по одному на клас)
    Dense(10, activation='softmax') # 'softmax' для класифікації
])

# Компіляція моделі
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Використовуємо, бо 'label_mode'='int'
    metrics=['accuracy']
)

model.summary()

# ---------------------------------
# 4. НАВЧАННЯ МОДЕЛІ
# ---------------------------------
print("\n--- Початок навчання Класифікатора (CNN) ---")

# (Опційно) EarlyStopping для зупинки, якщо точність перестала рости
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=30, # 30 епох повинно бути достатньо, EarlyStopping зупинить раніше
    callbacks=callbacks
)

print("--- Навчання завершено ---")

# ---------------------------------
# 5. ЗБЕРЕЖЕННЯ МОДЕЛІ
# ---------------------------------
MODEL_SAVE_PATH = 'learning/classifier_model.keras' # Новий формат збереження
model.save(MODEL_SAVE_PATH)

print(f"--- ✅ Модель класифікатора збережено в {MODEL_SAVE_PATH} ---")

2025-11-12 14:14:48.045011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 14:14:48.619717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-12 14:14:50.861177: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


--- Завантаження даних ---
Found 230802 files belonging to 10 classes.


I0000 00:00:1762956918.903751     426 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3554 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 99001 files belonging to 10 classes.
--- Дані успішно завантажено ---
Класи: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


/home/yesman/tfvenv/lib/python3.12/site-packages/keras/src/layers/preprocessing/data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

  10/3607 ━━━━━━━━━━━━━━━━━━━━ 46s 13ms/step - accuracy: 0.1628 - loss: 2.2630

I0000 00:00:1762956936.497274     512 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3604/3607 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9039 - loss: 0.3080

2025-11-12 14:17:02.732982: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_851', 8 bytes spill stores, 8 bytes spill loads



3607/3607 ━━━━━━━━━━━━━━━━━━━━ 162s 44ms/step - accuracy: 0.9452 - loss: 0.1797 - val_accuracy: 0.9645 - val_loss: 0.1207
Epoch 2/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - accuracy: 0.9694 - loss: 0.0991 - val_accuracy: 0.9699 - val_loss: 0.1049
Epoch 3/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - accuracy: 0.9740 - loss: 0.0827 - val_accuracy: 0.9683 - val_loss: 0.1067
Epoch 4/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - accuracy: 0.9765 - loss: 0.0734 - val_accuracy: 0.9653 - val_loss: 0.1256
Epoch 5/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.9784 - loss: 0.0672 - val_accuracy: 0.9699 - val_loss: 0.1058
Epoch 6/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - accuracy: 0.9796 - loss: 0.0623 - val_accuracy: 0.9701 - val_loss: 0.1121
Epoch 7/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.9810 - loss: 0.0572 - val_accuracy: 0.9700 - val_loss: 0.1270
Epoch 8/30
3607/3607 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.9823 - loss: 0.0